In [33]:
!pip install keras-tuner


### **Importing required Libraries**

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


### **Importing the dataset**

In [ ]:
fashion_mist = keras.datasets.fashion_mnist

In [ ]:
(train_images,train_labels),(test_images,test_labels) = fashion_mist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### **Scaling the image down**

In [ ]:
train_images = train_images /255.0
test_images = test_images /255.0

In [ ]:
train_images[0].shape     #Checking the shape

(28, 28)

### **Reshaping the image as a proper 3D image**

In [ ]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

## **Model Building**

In [ ]:
def build_model(hp):
  model = keras.Sequential([
        keras.layers.Conv2D(
            filters = hp.Int('conv_1_filter', min_value = 32, max_value = 128, step = 16),
            kernel_size = hp.Choice('conv_1_kernel', values =[3,5]),
            activation ='relu',
            input_shape =(28,28,1)        
          ),
          keras.layers.Conv2D(
            filters = hp.Int('conv_2_filter', min_value = 32, max_value = 64, step = 16),
            kernel_size = hp.Choice('conv_2_kernel', values =[3,5]),
            activation ='relu',    
          ),
          keras.layers.Flatten(),
          keras.layers.Dense(
              units = hp.Int('dense_1_units', min_value = 32, max_value = 128, step = 16),
              activation = 'relu'
          ),
          keras.layers.Dense(10, activation= 'softmax') # Output Layer
  ])

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values= [1e-2,1e-3])),
      loss = 'sparse_categorical_crossentropy',
      metrics = ['accuracy'])
  return model

### **Importing HyperParameter Engine**

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
tuner_search = RandomSearch(build_model,
                            objective = 'val_accuracy',
                            max_trials = 5, directory = 'output', project_name = "Mnist Fashion")

## **Training the Model**

In [ ]:
tuner_search.search(train_images,train_labels,epochs =3, validation_split =0.1)

Trial 6 Complete [00h 07m 17s]
val_accuracy: 0.9088333249092102

Best val_accuracy So Far: 0.9101666808128357
Total elapsed time: 01h 23m 22s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model = tuner_search.get_best_models(num_models=1)[0]

## **Displaying the Result**

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2150512   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,269,386
Trainable params: 2,269,386
Non-trainable params: 0
_________________________________________________________________
